In [37]:
from google.colab import drive

In [38]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!ls /content/drive/MyDrive/Amex/Predictions/

exponential_ensemble.csv
mean_submission.csv
test_catboost_dart_baseline_5fold_seed42_tmp2.csv
test_catboost_dart_baseline_5fold_seed42_tmp.csv
test_ensemble.csv
test_lgbm_baseline_5fold_seed42.csv
test_lgbm_dart_baseline_5fold_blend_tmp.csv
test_lgbm_dart_baseline_5fold_seed42.csv
test_lgbm_dart_baseline_5fold_seed42_tmp.csv
test_lgbm_dart_baseline_5fold_seed52_tmp.csv
test_lgbm_dart_baseline_5fold_seed62_tmp.csv
test_xgb_dart_baseline_5fold_seed42_tmp.csv


In [40]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

In [57]:
sub1 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/test_lgbm_dart_baseline_5fold_seed42_tmp.csv')
sub2 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/test_lgbm_dart_baseline_5fold_seed52_tmp.csv')
sub3 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/test_lgbm_dart_baseline_5fold_seed62_tmp.csv')
sub4 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/test_catboost_dart_baseline_5fold_seed42_tmp.csv')
sub5 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/test_catboost_dart_baseline_5fold_seed42_tmp2.csv')
sub6 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/exponential_ensemble.csv')
sub7 = pd.read_csv('/content/drive/MyDrive/Amex/Predictions/mean_submission.csv')

In [58]:
sub1 = sub1.rename(columns={'prediction': 'prediction1'})
sub2 = sub2.rename(columns={'prediction': 'prediction2'})
sub3 = sub3.rename(columns={'prediction': 'prediction3'})
sub4 = sub4.rename(columns={'prediction': 'prediction4'})
sub5 = sub5.rename(columns={'prediction': 'prediction5'})
sub6 = sub6.rename(columns={'prediction': 'prediction6'})
sub7 = sub7.rename(columns={'prediction': 'prediction7'})

In [59]:
sub = sub1.merge(sub2, how = 'inner', on = 'customer_ID').merge(sub3, how = 'inner', on = 'customer_ID').merge(sub4, how = 'inner', on = 'customer_ID').merge(sub5, how = 'inner', on = 'customer_ID').merge(sub6, how = 'inner', on = 'customer_ID').merge(sub7, how = 'inner', on = 'customer_ID')

In [60]:
W = [0.25, 0.25, 0.25, 0.20, 0.05, 0.025, 0.025]
sumW = sum(W)
W = [i/sumW for i in W]
W

[0.23809523809523814,
 0.23809523809523814,
 0.23809523809523814,
 0.19047619047619052,
 0.04761904761904763,
 0.023809523809523815,
 0.023809523809523815]

In [61]:
sub['prediction'] = sub['prediction1'] * W[0] + sub['prediction2'] * W[1] + sub['prediction3'] * W[2] + sub['prediction4'] * W[3]+ sub['prediction5'] * W[4] + sub['prediction6'] * W[5] + sub['prediction7'] * W[6]

In [62]:
sub.head()

,customer_ID,prediction1,prediction2,prediction3,prediction4,prediction5,prediction6,prediction7,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.032062,0.033367,0.033319,0.021244,0.020656,0.193998,0.005309,0.033287
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000884,0.000785,0.000894,0.001400,0.001088,0.054909,-0.017873,0.001811
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.033736,0.029986,0.030648,0.041785,0.038574,0.197795,0.007146,0.037145
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.179131,0.173700,0.182536,0.159147,0.210239,0.317868,0.082964,0.177337
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.889152,0.892144,0.880784,0.857692,0.867270,0.702775,0.427659,0.865412


In [63]:
sub = sub[['customer_ID', 'prediction']]
sub.to_csv('/content/drive/MyDrive/Amex/Predictions/test_ensemble.csv', index = False)

In [64]:
sub.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.033287
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001811
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.037145
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.177337
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.865412
